# COSMIC

### TODO

 * Get downloads working -- it's kinda crazy

In [16]:
from retraction_audit.audit import RetractionLookup
import os

os.environ["RETRACTION_WATCH_EMAIL"] = "james.sharpsteen@gmail.com"
os.environ["NCBI_AUTH_EMAIL"] = "james.sharpsteen@gmail.com"

lookup = RetractionLookup(True)

In [61]:
from wags_tails import CustomData

data_files = {}
for data_type in (
    "fusion", "structuralvariants", "resistancemutations",
    "noncodingvariants", "breakpoints", "hallmarksofcancer",
    "completetargetedscreensmutant", "genomescreensmutant",
    "mutantcensus"
):
    data_fetcher = CustomData(
        "cosmic",
        "tsv",
        lambda: "v99",
        lambda version, outfile_path: None,
        file_name=f"cosmic_{data_type}_grch38"
    )
    file = data_fetcher.get_latest(from_local=True)[0]
    data_files[data_type] = file

In [62]:
import polars as pl
pmids = []

for data_type, data_file in data_files.items():
    csv = pl.read_csv(
        data_file, 
        separator="\t", 
        ignore_errors=True
    )
    pmids += list(csv["PUBMED_PMID"].cast(str).unique())
pmids = list(set(pmids))

In [63]:
retracted_ev = []
for pmid in pmids:
    retraction_record = lookup.get_retraction_by_pmid(str(pmid))
    if retraction_record:
        retracted_ev.append(retraction_record)

In [64]:
print(len(retracted_ev))
from tools import store_results, get_latest_results
store_results("COSMIC", len(retracted_ev), retracted_ev, len(pmids))
get_latest_results()

15


[('CIViC', 1, 3667, '2024-03-15T00:03:17.308166+00:00'),
 ('DGIDB', 20, 11403, '2024-03-15T00:08:13.144921+00:00'),
 ('PMKB', 0, 1562, '2024-03-15T01:45:21.760435+00:00'),
 ('PharmGKB', 2, 17150, '2024-03-15T13:35:38.913493+00:00'),
 ('DOKB', 2, 4318, '2024-03-15T14:38:21.085659+00:00'),
 ('MOAlmanac', 0, 166, '2024-03-17T00:31:14.361317+00:00'),
 ('Gene2Phenotype', 4, 8369, '2024-03-19T01:35:50.498899+00:00'),
 ('COSMIC', 15, 20600, '2024-03-19T13:15:43.634143+00:00')]

In [65]:
retracted_ev

[RetractionData(pmid='30237408', doi='10.1038/s41419-018-0978-y', article_title='miR-665 promotes hepatocellular carcinoma cell migration, invasion, and proliferation by decreasing Hippo signaling through targeting PTPRB', journal='Cell Death & Disease', authors=['Yuanchang Hu', 'Chao Yang', 'Shikun Yang', 'Feng Cheng', 'Jianhua Rao', 'Xuehao Wang'], retraction_watch_refs=[], retraction_doi='10.1038/s41419-023-05654-x', retraction_pmid='36759596', retraction_reasons=['Duplication of Image', 'Informed/Patient Consent - None/Withdrawn', 'Unreliable Data', 'Unreliable Results'], retraction_type=<FlagType.RETRACTED: 'Retraction'>),
 RetractionData(pmid='24968753', doi='10.3892/ijmm.2014.1823', article_title='Downregulation of ROSâ\x80\x91FIG inhibits cell proliferation, colonyâ\x80\x91formation, cell cycle progression, migration and invasion, while inducing apoptosis in intrahepatic cholangiocarcinoma cells', journal='International Journal of Molecular Medicine', authors=['Gang Deng', 'Che